## Fly LSH
The fly olfactory circuit generates a “tag” for each odor, which is a set of neurons that fire when that odor is presented (1). This tag is critical for learning behavioral responses to different odors. For example, if a reward (e.g., sugar water) or a punishment (e.g., electric shock) is associated with an odor, that odor becomes attractive (a fly will approach the odor) or repulsive (a fly will avoid the odor), respectively. The tags assigned to odors are sparse—only a small fraction of the neurons that receive odor information respond to each odor (3–5)—and nonoverlapping: Tags for two randomly selected odors share few, if any, active neurons, so that different odors can be easily distinguished. - [A neural algorithm for a fundamental computing problem](https://science.sciencemag.org/content/sci/358/6364/793.full.pdf)
<img src="../images/fruitfly_fig_1.png" alt="Fly olfactory circuit" width="600"/>
<center>Figure A: Schematic of the fly olfactory circuit</center>

**Acronyms**
* ORNs - odorant receptor neurons
* PNs - projection neurons/glomeruli
* KCs - Kenyon cells
* APL - anterior paired lateral neuron

In [87]:
import numpy as np

class FlyLSH(object):
    def __init__(self, n=4, t=32, seed=1, rand_ratio=0.10, apl_ratio=0.05):
        """ FlyLSH is a locality sensitive hashing base off of fly olfactory circuit
        
        Class was inspired by the work at: https://github.com/pantsing/flylsh
        
        Args:
            n (int): number of grams, default is 4
            t (int): length of the output tag vector, default is 32
            seed (int): used for repeatable psuedo random generator, default is 1
            rand_ratio (float): between 0 and 1, ratio of PNs for each KCs, default is 0.10
            apl_ratio (float): between 0 and 1, ratio of winner KCs to be kept, default is 0.05
        
        """
        # Save passed in parameters
        self.n = n
        self.t = t
        self.rand_ratio = rand_ratio
        self.apl_ratio = apl_ratio
        self.seed = seed
        
        # Calculate number of odorant receptor neuron (ORN) and Kenyon cells (KCs)
        self.bits_in_byte = 8
        self.n_orns = n * self.bits_in_byte
        self.n_kcs = self.bits_in_byte * self.t
        
        # Set random seed (allows for repeatable analysis)
        np.random.seed(seed)
        
        # Set binary random projection matrix
        self.m = (np.random.rand(self.n_orns, self.n_kcs) <= rand_ratio).astype(np.uint8)
        
        # Set number of Kenyon cells to keep for anterior paired lateral (APL) neuron
        self.n_apl = int(np.ceil(self.n_kcs * self.apl_ratio))
        
        # Set mask used for feature vector
        self.mask = np.array([1<<i for i in range(self.bits_in_byte)], dtype=np.uint8)

    def tag(self, text):
        """ Creates the odor tag for the given string
        
        Args:
            text (str): collection of characters like a sentence
        
        Return:
            ndarray: 1-D numerical vector that represents the odor
        
        """
        # Initial step is to create a feature vector
        vector = self.vector(text)
        
        # Step 1: center the mean
        self.pns = vector - np.mean(vector)
        
        # Step 2: random projection
        self.kcs = self.pns.dot(self.m)
        
        # Step 3: Winner-take-all (WTA)
        top_indices = self.kcs.argsort()[-self.n_apl:]
        top_values = self.kcs[top_indices]
        self.kcs = np.zeros(self.kcs.shape)
        self.kcs[top_indices] = top_values
        
        # Convert WTA's Kenyon cells into desired tag length
        shape = (self.t, self.bits_in_byte)
        return (self.kcs > 0).reshape(shape).dot(self.mask)
    
    def vector(self, text):
        """ Creates a numerical vector from a string of text
        
        Args:
            text (str): collection of characters like a sentence
        
        Return:
            ndarray: 1-D numerical vector
        
        """
        text = np.array(list(text.encode()), dtype=np.int64)
        ngrams = self.ngram(text, self.n)
        vector = np.clip(ngrams[:,:,np.newaxis] & self.mask, 0, 1)
        return vector.sum(axis=0).flatten()
    
    def ngram(self, vector, n):
        """ Simple ngram generator using numpy
        
        Args:
            vector (ndarray): numerical vector
            n (int): number of grams
        
        Return:
            ndarray: numerical ngram vector with shape (*vector.shape, n)
        
        """
        
        shape = vector.shape[:-1] + (vector.shape[-1] - n + 1, n)
        strides = vector.strides + (vector.strides[-1],)
        return np.lib.stride_tricks.as_strided(vector, shape, strides)
        
    def compare(self, tag_1, tag_2):
        """ Simple comparison using XOR bit counts
        
        Args:
            tag_1 (ndarray): first odor tag
            tag_2 (ndarray): second odor tag
        
        Return:
            ndarray: approximate score, lower is better
        
        """
        return sum([bin(x).count('1') for x in tag_1 ^ tag_2])
    
    def to_hash(self, tag):
        """ Converts tag into a hash
        
        Args:
            tag (ndarray): vector representation of odor
        
        Return:
            str: string representation of odor
        
        """
        return ''.join('{:02X}'.format(x) for x in tag)
    
    def to_tag(self, hash_str):
        """ Converts hash into a tag
        
        Args:
            hash_str (str): string representation of odor
        
        Return:
            ndarray: vector representation of odor
        
        """
        vector = [int(hash_str[i:i+2], 16) for i in range(0, len(hash_str), 2)]
        return np.array(vector, dtype=np.uint8)

In [88]:
fly_lsh = FlyLSH()

docs = [
    'this is a test phrase',
    'this is a test phrass',
    'this is one of test phrases',
    'different test phrase'
]
    
tags = []
for doc in docs:
    tags.append(fly_lsh.tag(doc))
    hash_str = fly_lsh.to_hash(tags[-1])
    print(f'flylsh of "{doc}": {hash_str}')

print(f'Comparison of "{docs[0]}" and "{docs[1]}": {fly_lsh.compare(tags[0], tags[1])}')
print(f'Comparison of "{docs[0]}" and "{docs[2]}": {fly_lsh.compare(tags[0], tags[2])}')
print(f'Comparison of "{docs[0]}" and "{docs[3]}": {fly_lsh.compare(tags[0], tags[3])}')

flylsh of "this is a test phrase": 0002002000208040000200000042000000001000000000000100001000001200
flylsh of "this is a test phrass": 0002002000208040000200000042000000001000000000000100001000001200
flylsh of "this is one of test phrases": 0002002000208040000200000042000000000000000000000100000002801200
flylsh of "different test phrase": 8002002000208040000200000000000000000000000000000000020002009202
Comparison of "this is a test phrase" and "this is a test phrass": 0
Comparison of "this is a test phrase" and "this is one of test phrases": 4
Comparison of "this is a test phrase" and "different test phrase": 10
